In [ ]:
import numpy as np
import torch
import keras
from keras.callbacks import EarlyStopping
from keras import regularizers
from tensorflow.keras.layers import Dense, BatchNormalization
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras.layers import Dense,Flatten
from keras.layers import MaxPooling2D
import pandas as pd
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Input,Conv2D,MaxPool2D,Flatten,Dense
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam ,RMSprop,Adagrad,Adadelta
import scipy.io as sio
import random
from sklearn.model_selection import train_test_split
import h5py
import warnings
warnings.filterwarnings('ignore')
# from __future__ import print_function

In [ ]:
def load_data():
    
    l0=2031
    l1=1896
    l2=1962
    l3=1963
    l4=2025

##################
   
#     data=sio.loadmat('/Users/apple/Documents/MATLAB/井中微地震监测/DATA_Augment_6.20/DATApy.mat')
#      data=sio.loadmat('/Users/apple/Documents/MATLAB/本科毕业论文/DATApy.mat')
    data1=sio.loadmat('/Users/apple/Documents/MATLAB/本科毕业论文/DATApy1.mat')
    data2=sio.loadmat('/Users/apple/Documents/MATLAB/本科毕业论文/DATApy2.mat')
    data3=sio.loadmat('/Users/apple/Documents/MATLAB/本科毕业论文/DATApy3.mat')
    data4=sio.loadmat('/Users/apple/Documents/MATLAB/本科毕业论文/DATApy4.mat')
    data5=sio.loadmat('/Users/apple/Documents/MATLAB/本科毕业论文/DATApy5.mat')

    data1 = data1['DATApy1']    # 大小为(x, 1001, 12, 3)
    data2 = data2['DATApy2']    # 大小为(x, 1001, 12, 3)
    data3 = data3['DATApy3']    # 大小为(x, 1001, 12, 3)
    data4 = data4['DATApy4']    # 大小为(x, 1001, 12, 3)
    data5 = data5['DATApy5']    # 大小为(x, 1001, 12, 3)
    
    # 将五个数组沿着第一个维度拼接
    data = np.concatenate((data1,data2, data3, data4, data5 ),axis=0)

#     data=h5py.File('/Users/apple/Documents/MATLAB/本科毕业论文/DATApy.mat','r')
    tr_num_1=l0
    tr_num_2=l1+tr_num_1
    tr_num_3=l2+tr_num_2
    tr_num_4=l3+tr_num_3
    tr_num_5=l4+tr_num_4

    tr_num_11=l0
    tr_num_22=l1
    tr_num_33=l2
    tr_num_44=l3
    tr_num_55=l4
    
#     x_train_1=data['DATApy'][0:tr_num_1]
#     x_train_2=data['DATApy'][tr_num_1:tr_num_2]
#     x_train_3=data['DATApy'][tr_num_2:tr_num_3]
#     x_train_4=data['DATApy'][tr_num_3:tr_num_4]
#     x_train_5=data['DATApy'][tr_num_4:tr_num_5]

    x_train_1=data[0:tr_num_1,:,:,:]
    x_train_2=data[tr_num_1:tr_num_2,:,:,:]
    x_train_3=data[tr_num_2:tr_num_3,:,:,:]
    x_train_4=data[tr_num_3:tr_num_4,:,:,:]
    x_train_5=data[tr_num_4:tr_num_5,:,:,:]
    
    y_train_1=[[1,0,0,0,0]]*tr_num_11
    y_train_2=[[0,1,0,0,0]]*tr_num_22
    y_train_3=[[0,0,1,0,0]]*tr_num_33
    y_train_4=[[0,0,0,1,0]]*tr_num_44
    y_train_5=[[0,0,0,0,1]]*tr_num_55


    
    x_train=np.concatenate((x_train_1,x_train_2,x_train_3,x_train_4,x_train_5))
    y_train=np.concatenate((y_train_1,y_train_2,y_train_3,y_train_4,y_train_5))
    
    return (x_train,y_train)
    
#     return (x_train,y_train),(x_test,y_test)

In [ ]:
def shuffle_data(data,ydata,seed=1.0):
    index=[i for i in range(len(ydata))]
    random.seed(seed)
    random.shuffle(index)
    data = [data[i] for i in index]
    ydata = [ydata[i] for i in index]
    return np.array(data),np.array(ydata)

In [ ]:
def plot_ntrc(data):
    dmax=data.max()
#     for i,data1 in enumerate(data):
#         plt.plot(data1[:]+i*dmax)
    for i,data1 in enumerate(data):
        for ii in [0, 1, 2]:
            if ii==0:
#                  plt.plot(data1[:,ii]+i*dmax,'mediumpurple')
                 plt.plot(data1[:,ii]+i*dmax,'r')

            if ii==1:
#                  plt.plot(data1[:,ii]+i*dmax,'lightcoral')
                 plt.plot(data1[:,ii]+i*dmax,'b')

            if ii==2:
#                  plt.plot(data1[:,ii]+i*dmax,'cadetblue')
                 plt.plot(data1[:,ii]+i*dmax,'g')



In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_accuracy = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('accuracy'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_accuracy['batch'].append(logs.get('val_accuracy'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('accuracy'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_accuracy['epoch'].append(logs.get('val_accuracy'))

    def loss_plot(self, loss_type):

        iters = range(len(self.losses[loss_type]))

        # acc
#         plt.figure(figsize=(7, 5))
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train accuracy')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_accuracy[loss_type], 'b', label='val accuracy')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.ylim([0.0, 1.05])
#         plt.legend(loc="best")
        plt.legend(loc='right')
        plt.tight_layout()
        plt.savefig('训练过程.png',dpi=1200)
        plt.show()

In [ ]:
class Detectevent():
#     def __init__(self):
#         (x_train,y_train),(x_test,y_test)=load_data()
        
#         (x,y)=load_data()
#         (x_train,x_test,y_train,y_test)=train_test_split(x,y,test_size=0.2)
#         (x_train,y_train)=shuffle_data(x_train,y_train)
#         (x_test,y_test)=shuffle_data(x_test,y_test)
        
#         self.x_train,self.y_train=shuffle_data(x_train,y_train)
#         self.x_test=x_test
#         self.y_test = y_test
#         print('shape',x_train.shape,y_train.shape)

#     def get_cnn(self):
#         inputs = Input((12, 1001, 3))
#         conv1 = Conv2D(16, kernel_size=(3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
#         conv1 = MaxPool2D(pool_size=(2,4))(conv1)
        
#         conv2 = Conv2D(32, kernel_size=(3, 3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
#         conv2 = MaxPool2D(pool_size=(2,2))(conv2)
#         conv2 = Flatten()(conv2)
        
#         dense=Dense(128,activation='relu')(conv2)
#         dense=Dense(5,activation='softmax')(dense)
        
#         model = Model(input = inputs, outputs = dense)
#         model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
#         return model

    def train(self):
        model=self.get_cnn()
        model.summary()
        model_checkpoint = ModelCheckpoint('network1.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
        model.fit(self.x_train,self.y_train, batch_size=4, nb_epoch=20, verbose=1,validation_split=0.25, 
                   shuffle=True, callbacks=[model_checkpoint])
    def test(self):
        model=load_model('network1.hdf5')
        y_predict=model.predict(self.x_test[0::3])
        print('predict:',y_predict)
        print('true:',self.y_test[0::3])

In [ ]:
(x,y)=load_data()
# x = np.transpose(x, (3, 2, 1, 0)) #通过h5py读取的数据维度排列调转了(使用h5py才使用)

In [ ]:
(x_train,x_test,y_train,y_test)=train_test_split(x,y,test_size=0.20,random_state=42)
(x_train,y_train)=shuffle_data(x_train,y_train)
(x_test,y_test)=shuffle_data(x_test,y_test)

In [ ]:
model=Detectevent()

In [ ]:
network = models.Sequential()
network.add(layers.Conv2D(filters=64, kernel_size=(3, 3),padding='valid', activation='relu', input_shape=(12, 1001,3),
                          name="conv1"))
# network.add(BatchNormalization(epsilon=0.0001, momentum=0.9))
network.add(layers.MaxPooling2D((2,2)))
network.add(layers.Dropout(0.5))
network.add(layers.Conv2D(filters=128, kernel_size=(3, 3),padding='valid', activation='relu',name="conv2"))
# network.add(BatchNormalization(epsilon=0.0001, momentum=0.9))
network.add(layers.MaxPooling2D((2, 2)))
network.add(layers.Dropout(0.5))

network.add(layers.Flatten())


network.add(layers.Dense(128, activation='relu',name="dense4"))

network.add(layers.Dense(64, activation='relu',name="dense5"))

network.add(layers.Dense(5, activation='softmax',name="dense6"))

network.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('network1.hdf5',
                             monitor='val_accuracy', save_weights_only=True,verbose=1,save_best_only=True, period=1,mode='max')
# 定义 EarlyStopping 回调函数
early_stopping = EarlyStopping(monitor='val_accuracy', patience=6)

history = LossHistory()
# callbacks=[history]

In [ ]:
network.fit(x_train,y_train,epochs=30,  batch_size=32,validation_data=(x_test, y_test),callbacks=[history],shuffle=True)

# network.fit(x_train,y_train,epochs=30,  batch_size=64,validation_data=(x_test, y_test),callbacks=[early_stopping ],shuffle=True)

In [ ]:
history.loss_plot('epoch')

In [ ]:
network.save('model/my_network_4.4.h5')

In [ ]:
score = network.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
network.summary()

# 评价精度

## 1、混淆矩阵

In [ ]:
import matplotlib
from sklearn.preprocessing import Normalizer
matplotlib.rc("font",family='SimHei') # 中文字体
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
# 禹行精选配色方案
# Blues
# BuGn
# Reds
# Greens
# Greys
# binary
# Oranges
# Purples
# BuPu
# GnBu
# OrRd
# RdPu

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens,#这个主题看着就干净~
                          normalize=True):
   
 
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Greens')

    plt.figure(figsize=(7, 7))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.3f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


#     plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.colorbar()
    plt.savefig('混淆矩阵.png',dpi=1000)
    plt.show()


In [ ]:
def plot_confuse_nor(model, x_val, y_val):
    predictions = network.predict_classes(x_val,batch_size=5)
    truelabel = y_val.argmax(axis=-1)   # 将one-hot转化为label
    conf_mat = confusion_matrix(y_true=truelabel, y_pred=predictions)
    conf_mat=Normalizer().fit_transform(conf_mat)
    plt.figure()
    plot_confusion_matrix(conf_mat, normalize=True,target_names=labels,title='Confusion Matrix')

In [ ]:
labels=['1','2','3','4','5']
plot_confuse_nor(model, x_test,y_test)

In [ ]:
def plot_confuse(model, x_val, y_val):
    predictions = network.predict_classes(x_val,batch_size=5)
    truelabel = y_val.argmax(axis=-1)   # 将one-hot转化为label
    conf_mat = confusion_matrix(y_true=truelabel, y_pred=predictions)
    plt.figure()
    plot_confusion_matrix(conf_mat, normalize=False,target_names=labels,title='Confusion Matrix')

In [ ]:
plot_confuse(model, x_test,y_test)

## 绘制ROC曲线

In [ ]:

df = pd.read_csv('/Users/apple/Desktop/本科毕业论文/类别.csv')
classes = df['标注类别名称'].unique()
print(classes)

In [ ]:
class_list = np.unique(df['标注类别名称'])

In [ ]:
n_class = len(class_list) # 测试集标签类别数
# palette = sns.hls_palette(n_class) # 配色方案 
palet = sns.hls_palette(n_class, l=.4, s=1)

# sns.palplot(palette)
sns.palplot(palet)


In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
n_classes=5
# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    y_score= network.predict(x_test)
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [ ]:
plt.figure(figsize=(7, 7))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = palet
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.4f})'
             ''.format(i+1, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')

plt.legend(loc="lower right")
plt.savefig('ROC曲线.png',dpi=1000)
plt.show()

In [ ]:
plt.figure(figsize=(7, 7))
plt.xlim(0, 0.04)
plt.ylim(0.96, 1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = palet
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.4f})'
             ''.format(i+1, roc_auc[i]))
ax = plt.gca()
ax.set_aspect(1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig('ROC曲线局部.png',dpi=1000)
plt.show()

## 绘制PR曲线

In [ ]:
#绘制多分类PR曲线
plt.figure(figsize=(7, 7))
from sklearn.metrics import precision_recall_curve
precision = dict()
recall = dict()
n_classes = 5
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_score[:, i])
plt.rcParams['font.family']=['Times New Roman']

lw = 2
colors = palet
for i, color in zip(range(n_classes), colors):
    plt.plot(precision[i], recall[i], color=color, lw=lw,
             label='PR curve of class {0}'.format(i+1))

plt.plot([1, 0], [0, 1], 'k--', lw=lw)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Precision recall curve for multi-class data')
plt.legend(loc="lower left")
ax = plt.gca()
ax.set_aspect(1)
plt.savefig('PR曲线.png',dpi=1000)
plt.show()

In [ ]:
plt.figure(figsize=(7, 7))
from sklearn.metrics import precision_recall_curve
precision = dict()
recall = dict()
n_classes = 5
for i in range(n_classes):
    precision[i], recall[i], thresholds = precision_recall_curve(y_test[:, i], y_score[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='Class {0}'.format(i+1), color=palet[i])

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

# 添加颜色条
sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=plt.Normalize(vmin=0, vmax=1))
sm.set_array(thresholds)
plt.colorbar(sm, label='Threshold')

plt.show()


In [ ]:
#绘制多分类PR曲线
plt.figure(figsize=(7, 7))
plt.xlim(0.93, 1)
plt.ylim(0.93, 1)
plt.rcParams['font.family']=['Times New Roman']

lw = 2
colors = palet
for i, color in zip(range(n_classes), colors):
    plt.plot(precision[i], recall[i], color=color, lw=lw,
             label='PR curve of class {0}'.format(i+1))

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Precision recall curve for multi-class data')
plt.legend(loc="lower left")
plt.savefig('PR曲线局部.png',dpi=1000)
plt.show()


$MSE=\frac{1}{N}\sum_{i=1}^{N}\left(Y_i-f\left(X_i\right)\right)^2$


# 预测错误分析

In [ ]:
import matplotlib.pyplot as plt

# 对测试集进行预测
y_pred = network.predict(x_test)
y_pred

In [ ]:
# 找到预测错误的样本
incorrect = np.nonzero(y_pred.argmax(axis=1) != y_test.argmax(axis=1))[0]
incorrect

In [ ]:
y_test[incorrect]

In [ ]:
y_pred[incorrect].argmax(axis=1)

In [ ]:
y_pred[incorrect]

In [ ]:
# 可视化预测错误的样本
L=incorrect
for ii in list(range(len(L))):
    plt.figure(ii)
    plot_ntrc(x_test[L[ii]])


# 类别分析

In [ ]:
L1=[x for x,y in enumerate(y_test) if y_test[x][0] == 1]
L2=[x for x,y in enumerate(y_test) if y_test[x][1] == 1]
L3=[x for x,y in enumerate(y_test) if y_test[x][2] == 1]
L4=[x for x,y in enumerate(y_test) if y_test[x][3] == 1]
L5=[x for x,y in enumerate(y_test) if y_test[x][4] == 1]
L=[L1 ,L2,L3, L4 ,L5]

In [ ]:
L=L2
for ii in list(range(300)):
    plt.figure(ii)
    plot_ntrc(x_test[L[ii]])



# 使用验证集选择阈值

In [ ]:
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np

In [ ]:
X_val=x_test
y_val=y_test
y_val_pred =network.predict(X_val)

In [ ]:
# # 网格搜索最佳阈值
# num_classes=5
# num_thresholds=20
# thresholds = np.linspace(0, 1, num_thresholds)
# best_thresholds = [0.0] * num_classes
# best_scores = [0.0] * num_classes
# for i in range(num_classes):
#     y_val_pred = network.predict(X_val)
#     val_scores = []
#     for t in thresholds:
#         y_val_pred_thresholded = (y_val_pred[:, i] >= t).astype(int)
#         val_score = f1_score(y_val[:, i], y_val_pred_thresholded)
#         val_scores.append(val_score)
#     best_index = np.argmax(val_scores)
#     best_thresholds[i] = thresholds[best_index]
#     best_scores[i] = val_scores[best_index]

In [ ]:
# 设置字体
plt.rcParams['font.family'] = 'Arial'

# 网格搜索最佳阈值
num_classes = 5
num_thresholds = 20
thresholds = np.linspace(0, 1, num_thresholds)
best_thresholds = [0.0] * num_classes
best_scores = [0.0] * num_classes

# 初始化画布
fig, ax = plt.subplots(figsize=(8, 8))

# 循环处理每个类别
for i in range(num_classes):
    y_val_pred = network.predict(X_val)
    val_scores = []
    for t in thresholds:
        y_val_pred_thresholded = (y_val_pred[:, i] >= t).astype(int)
        val_score = f1_score(y_val[:, i], y_val_pred_thresholded)
        val_scores.append(val_score)
    
    best_index = np.argmax(val_scores)
    best_thresholds[i] = thresholds[best_index]
    best_scores[i] = val_scores[best_index]
    
   
     # 绘制当前类别的阈值搜索过程
    line_color = sns.color_palette()[i % len(sns.color_palette())]
    ax.plot(thresholds, val_scores, linewidth=2, label='F1 score of C{0}'.format(i+1), color=line_color)
    ax.scatter(best_thresholds[i], best_scores[i], s=50, color=line_color, edgecolors='black', zorder=3)
    ax.axvline(x=best_thresholds[i], linestyle='--', color=line_color, alpha=0.5)
#     ax.text(best_thresholds[i], best_scores[i], " C{} {:.3f}".format(i+1,best_thresholds[i]), fontsize=12, ha='center', va='top')
    
    
# 添加全局标题和坐标轴标签
plt.title('Threshold Selection', fontsize=18, fontweight='bold')
plt.xlabel('Threshold', fontsize=14)
plt.ylabel('F1 Score', fontsize=14)
plt.xlim(0.0, 1.0)
plt.ylim(0.9, 1.01)   

# 添加图例和背景网格
ax.legend(frameon=True, facecolor='white', framealpha=0.9, fontsize=12)
ax.grid(color='lightgray', linestyle='-', linewidth=0.5)

# 设置坐标轴标签的字体和刻度
plt.tick_params(axis='both', which='major', labelsize=12)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0.9, 1.005, 0.01))

# 调整图像边界和填充
plt.tight_layout(pad=3.0, rect=[0.03, 0.03, 0.97, 0.97])
plt.savefig('阈值选择.png',dpi=1000)
# 显示图表
plt.show()


In [ ]:
best_thresholds

#  减维可视化

In [ ]:
middle = Model(inputs=network.input,outputs=network.get_layer('dense5').output)
# result = middle.predict(x_test)[0]
result = middle.predict(x_test)
print(result.shape)

In [ ]:

df = pd.read_csv('/Users/apple/Desktop/本科毕业论文/类别.csv')

In [ ]:
df.head()

In [ ]:
classes = df['标注类别名称'].unique()
print(classes)

In [ ]:
class_list = np.unique(df['标注类别名称'])

In [ ]:
import seaborn as sns
marker_list = ['<', 'o', 'v', '^','p']

In [ ]:
class_list

In [ ]:
n_class = len(class_list) # 测试集标签类别数
# palette = sns.hls_palette(n_class) # 配色方案 
palet = sns.hls_palette(n_class, l=.4, s=1)

# sns.palplot(palette)
sns.palplot(palet)


In [ ]:
# import random
# random.seed(1234)
# random.shuffle(marker_list)
# random.shuffle(palet)

# T-SNE二维可视化

In [ ]:
network=load_model('model/my_network_4.3.h5')

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, n_iter=1000)
X_tsne_2d = tsne.fit_transform(result)


In [ ]:
X_tsne_2d.shape

In [ ]:
L1=[x for x,y in enumerate(y_test) if y_test[x][0] == 1]
L2=[x for x,y in enumerate(y_test) if y_test[x][1] == 1]
L3=[x for x,y in enumerate(y_test) if y_test[x][2] == 1]
L4=[x for x,y in enumerate(y_test) if y_test[x][3] == 1]
L5=[x for x,y in enumerate(y_test) if y_test[x][4] == 1]
L=[L1 ,L2,L3, L4 ,L5]

In [ ]:
L=[L1 ,L2,L3, L4 ,L5]

In [ ]:
# 不同的 符号 表示 不同的 标注类别
show_feature = '标注类别名称'

In [ ]:
plt.figure(figsize=(14, 14))
for idx, fruit in enumerate(class_list): # 遍历每个类别
    # 获取颜色和点型
    color = palet[idx]
    marker = marker_list[idx%len(marker_list)]
    
    # 找到所有标注类别为当前类别的图像索引号
    indices = L[idx]

    plt.scatter(X_tsne_2d[indices, 0], X_tsne_2d[indices, 1], color=color, marker=marker, label=fruit, alpha=0.2,s=150)

plt.legend(fontsize=16, markerscale=1, bbox_to_anchor=(1, 1),framealpha=0.5)
plt.xticks([])
plt.yticks([])
plt.savefig('语义特征t-SNE二维降维可视化.png', dpi=1200) # 保存图像
plt.show()

In [ ]:
L5


In [ ]:

a=198


plot_ntrc(x_train[a])
print('label:',y_train[a])

In [ ]:
plt.figure(figsize=(14, 14))
for idx, fruit in enumerate(class_list): # 遍历每个类别
    # 获取颜色和点型
    color = palet[idx]
    marker = marker_list[idx%len(marker_list)]
    
    # 找到所有标注类别为当前类别的图像索引号
    indices = L[idx]

    plt.scatter(X_tsne_2d[indices, 0], X_tsne_2d[indices, 1], color=color, marker=marker, label=fruit, alpha=0.3,s=150)
plt.scatter(X_tsne_2d[a, 0], X_tsne_2d[a, 1], color='r', marker='x', label='c_point', alpha=1,s=600)
plt.legend(fontsize=16, markerscale=1, bbox_to_anchor=(1, 1),framealpha=0.5)
plt.xticks([])
plt.yticks([])
plt.savefig('语义特征t-SNE二维降维可视化2.pdf', dpi=1200) # 保存图像
plt.show()

# PCA二维可视化

In [ ]:
from sklearn.decomposition import PCA
x = result
pca = PCA(n_components=2)  # 加载PCA算法，设置降维后主成分数目为2
reduced_x = pca.fit_transform(x)  # 对样本进行降维

In [ ]:
reduced_x.shape

In [ ]:
plt.figure(figsize=(14, 14))
for idx, fruit in enumerate(class_list): # 遍历每个类别
    # 获取颜色和点型
    color = palet[idx]
    marker = marker_list[idx%len(marker_list)]
    
    # 找到所有标注类别为当前类别的图像索引号
    indices = L[idx]

    plt.scatter(reduced_x[indices, 0], reduced_x[indices, 1], color=color, marker=marker, label=fruit,  alpha=0.5,s=50)

plt.legend(fontsize=16, markerscale=3, bbox_to_anchor=(1, 1),framealpha=0.5)
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.savefig('语义特征PCA二维降维可视化.pdf', dpi=1200) # 保存图像
plt.show()

# 可视化卷积层

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
network.layers[0] 

In [ ]:

# 可视化卷积层
conv_layer = network.layers[0]  # 获取卷积层
filters = conv_layer.get_weights()[0]  # 获取卷积层的权重
filters = np.transpose(filters, (3, 0, 1, 2))  # 调整维度顺序，使其适合imshow函数

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=8, figsize=(8, 4))
for i, ax in enumerate(axs.flatten()):
    if i < 32:
        ax.imshow(filters[i, :, :, 0], cmap='gray')
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.savefig('卷积层.png', dpi=1200) # 保存图像
plt.show()


In [ ]:
# 可视化卷积层
conv_layer = network.layers[3]  # 获取卷积层
filters = conv_layer.get_weights()[0]  # 获取卷积层的权重
filters = np.transpose(filters, (3, 0, 1, 2))  # 调整维度顺序，使其适合imshow函数

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=8, figsize=(8, 4))
for i, ax in enumerate(axs.flatten()):
    if i < 32:
        ax.imshow(filters[i, :, :, 0], cmap='gray')
        ax.axis('off')
    else:
        ax.axis('off')
plt.show()